In [1]:
%cd ../..

/home/eli/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target_actor = fep_control.MountainCarActor(state_dim=3, batch_shape=(1,), trainable=False)
target_observer = fep_control.GenerativeObserver(observation_dim=2, batch_shape=(1,), trainable=False)
target = compose(target_observer, target_actor)

proposal_actor = fep_control.RecognitionActor(state_dim=3, action_dim=1, observation_dim=2, batch_shape=(1,),
                                              discrete_actions=False, name='MountainCarActor', trainable=True)
proposal_encoder = fep_control.RecognitionEncoder(state_dim=3, observation_dim=2, batch_shape=(1,), trainable=True)
proposal = compose(proposal_encoder, proposal_actor)

agent = importance.propose(target, proposal)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-2, episodes=10, episode_length=1000, dream=False)

06/03/2019 18:03:55 ELBO=-2.84041953e+04 at episode 0 of length 1000
06/03/2019 18:04:03 ELBO=-2.21199570e+04 at episode 1 of length 1000
06/03/2019 18:04:11 ELBO=-2.21296270e+04 at episode 2 of length 1000
06/03/2019 18:04:19 ELBO=-2.14250391e+04 at episode 3 of length 1000
06/03/2019 18:04:27 ELBO=-2.04588359e+04 at episode 4 of length 1000
06/03/2019 18:04:36 ELBO=-2.01921172e+04 at episode 5 of length 1000
06/03/2019 18:04:44 ELBO=-2.38422617e+04 at episode 6 of length 1000
06/03/2019 18:04:53 ELBO=-2.01716992e+04 at episode 7 of length 1000
06/03/2019 18:05:01 ELBO=-2.03455820e+04 at episode 8 of length 1000
06/03/2019 18:05:09 ELBO=-2.01703555e+04 at episode 9 of length 188


In [7]:
log_weight

tensor([-20170.3555], device='cuda:0', grad_fn=<AddBackward0>)

In [8]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[-0.0317,  0.1386,  0.0600]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[3.1996]], device='cuda:0', grad_fn=<AddBackward0>))

In [10]:
log_weight

tensor([-10868.5566], device='cuda:0', grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')